In [0]:
# configure environment to run this notebooks
import uuid
import yt.wrapper as yt

username = yt.get_user_name()
if yt.exists(f"//sys/users/{username}/@user_info/home_path"):
    # prepare working directory on distributed file system
    user_info = yt.get(f"//sys/users/{yt.get_user_name()}/@user_info")
    homedir = user_info["home_path"]
    # find avaliable vm presets
    cpu_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("cpu")] or ["default"]
    h100_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("h100")]
    h100_8_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("h100-8")]
    workdir = f"{homedir}/tmp/demo_workdir/{uuid.uuid4().hex}"
else:
    cpu_pool_trees = ["default"]
    h100_pool_trees = ["gpu_h100"]
    h100_8_pool_trees = ["gpu_h100"]
    workdir = f"//tmp/examples/{uuid.uuid4().hex}"

yt.create("map_node", workdir, recursive=True, ignore_existing=True)
print("Current working directory:", workdir)

Current working directory: //home/equal_amethyst_vulture/tmp/demo_workdir/f55c44d313ec41bd9db05ac3b0769fa3


You can use YTsaurus client without passing a proxy and a token.
The client implicitly uses the token from the YT_TOKEN secret and works with the current cluster.

Let's get attributes from `//sys` directory.

In [2]:
from yt import wrapper as yt

yt.get("//sys/@")

{'id': '1-22-24dd025a-31bf68cd', 'type': 'sys_node', 'builtin': false, 'sequoia': false, 'ref_counter': 1, 'ephemeral_ref_counter': 0, 'weak_ref_counter': 0, 'foreign': false, 'native_cell_tag': 9437, 'revision': 2701534465859, 'attribute_revision': 2160368750243, 'content_revision': 2701534465859, 'inherit_acl': true, 'acl': [{'action': 'allow', 'subjects': ['tenants'], 'permissions': ['read'], 'inheritance_mode': 'object_only'}, {'action': 'allow', 'subjects': ['robot-ui'], 'permissions': ['read'], 'inheritance_mode': 'object_only'}], 'owner': 'root', 'effective_acl': YsonEntity, 'user_attribute_keys': ['enable_safe_mode', 'ui_config'], 'opaque_attribute_keys': YsonEntity, 'user_attributes': YsonEntity, 'life_stage': 'creation_committed', 'estimated_creation_time': YsonEntity, 'parent_id': 'ffffffff-ffffffff-24dd012f-ef1d693a', 'external': false, 'locks': YsonEntity, 'lock_count': 0, 'lock_mode': 'none', 'path': YsonEntity, 'key': 'sys', 'effective_expiration': YsonEntity, 'creation_

Create ytsaurus client, create table and sort it.

In [4]:
client = yt.YtClient(config=yt.default_config.get_config_from_env())

In [5]:
records = [{"key": "a", "value": 3}, {"key": "b", "value": 2}, {"key": "c", "value": 1}]
table_path = f"{workdir}/table_to_be_sorted"
client.write_table(f"{workdir}/table_to_be_sorted", input_stream=records, force_create=True)

In [6]:
client = yt.YtClient(config=yt.default_config.get_config_from_env())

In [7]:
print([r for r in client.read_table(table_path)])

[{'key': 'a', 'value': 3}, {'key': 'b', 'value': 2}, {'key': 'c', 'value': 1}]


In [8]:
client.run_sort(table_path, table_path, sort_by="value")

2025-06-19 18:35:12,936	INFO	Operation started: https://playground.tracto.ai/playground/operations/42dba941-bf9fa562-24dd03e8-34cc49d7/details


2025-06-19 18:35:12,972	INFO	( 0 min) operation 42dba941-bf9fa562-24dd03e8-34cc49d7 starting


2025-06-19 18:35:13,505	INFO	( 0 min) operation 42dba941-bf9fa562-24dd03e8-34cc49d7 initializing


2025-06-19 18:35:14,161	INFO	( 0 min) operation 42dba941-bf9fa562-24dd03e8-34cc49d7: running=0     completed=0     pending=1     failed=0     aborted=0     lost=0     total=1     blocked=0    


2025-06-19 18:35:14,743	INFO	( 0 min) operation 42dba941-bf9fa562-24dd03e8-34cc49d7: running=0     completed=1     pending=0     failed=0     aborted=0     lost=0     total=1     blocked=0    


2025-06-19 18:35:15,283	INFO	( 0 min) operation 42dba941-bf9fa562-24dd03e8-34cc49d7 completed


In [9]:
print([r for r in client.read_table(table_path)])

[{'value': 1, 'key': 'c'}, {'value': 2, 'key': 'b'}, {'value': 3, 'key': 'a'}]
